In [1]:
from platform import python_version
print('Python version:', python_version())

Python version: 3.10.6


In [2]:
!nvidia-smi

Mon Mar 13 17:15:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 30%   31C    P8    N/A /  75W |    134MiB /  4096MiB |     18%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
import csv
import zipfile
import math
import random
import collections
import numpy as np
import tensorflow as tf
import pandas as pd
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from PIL import Image
from collections import Counter
import matplotlib
import matplotlib.gridspec as gridspec
import json
%matplotlib inline

from nltk.translate.bleu_score import corpus_bleu
import nltk

2023-03-13 17:15:45.791969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 17:15:46.530478: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-13 17:15:47.886612: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-13 17:15:47.886720: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

## Dataset

[Dowload](https://nlp.stanford.edu/projects/nmt/):

* English vocabulary: [`vocab.50K.en`](https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.en)

### Loading the Datasets and Building the Vocabulary

First, we build the vocabulary dictionaries for the source and target (English) language. 
The vocabularies are found in the file `vocab.50K.en`(English).

In [4]:
# Word string -> ID mapping
dictionary = dict()

vocabulary_size = len(dictionary)
with open('data/vocab.50K.en', encoding='utf-8') as f:
    for line in f:
        # disregard the new line aka `\n`
        dictionary[line[:-1]] = len(dictionary)
        
vocabulary_size = len(dictionary)
reverse_dictionary = dict(zip(dictionary.values(),dictionary.keys()))

print('Dictionary:', list(dictionary.items())[:10], end = '\n')
print('Reverse dictionary:', list(reverse_dictionary.items())[:10], end = '\n')
print('Vocabulary size: ', vocabulary_size, end = '\n')


Dictionary: [('<unk>', 0), ('<s>', 1), ('</s>', 2), ('the', 3), (',', 4), ('.', 5), ('of', 6), ('and', 7), ('to', 8), ('in', 9)]
Reverse dictionary: [(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, 'the'), (4, ','), (5, '.'), (6, 'of'), (7, 'and'), (8, 'to'), (9, 'in')]
Vocabulary size:  50000


### Loading data
Here we load the data from the dataset.csv file (generated in the other script)

In [5]:
dataset = pd.read_csv('data/dataset.csv')

### Data pre-processing
Transform to lower, remove the new line and the punctuation

In [6]:
wt = nltk.tokenize.WhitespaceTokenizer()

for column in dataset.columns:
    dataset[column] = dataset[column].str.lower() 
    dataset[column] = dataset[column].str.replace(',', ' ,')  \
                                     .str.replace('.',' .')   \
                                     .str.replace('?',' ?')   \
                                     .str.replace('\n',' ')
    dataset[column] = dataset[column].apply(wt.tokenize)


/tmp/ipykernel_9509/3017365748.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('.',' .')   \
/tmp/ipykernel_9509/3017365748.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('?',' ?')   \


In [7]:
dataset.head()

,question,answer
0,"[what's, the, purpose, of, life, ?]","[idk, ., try, to, figure, out, how, to, be, ha..."
1,"[what's, the, purpose, of, life, ?]","[the, purpose, of, life, is, to, produce, comp..."
2,"[what's, the, purpose, of, life, ?]","[the, same, purpose, that, the, first, replica..."
3,"[what's, the, purpose, of, life, ?]","[have, fun, and, help, others, ,, change, the,..."
4,"[i've, tried, to, quit, smoking, ,, this, is, ...","[""i, smoked, for, 12y, then, stopped, ., it, a..."


### Data analysis
Mean sentence length and standard deviation of sentence length

In [40]:
print('(Questions) Average sentence length: ', dataset['question'].str.len().mean())
print('(Questions) Standard deviation of sentence length: ', dataset['question'].str.len().std())

print('(Answers) Average sentence length: ', dataset['answer'].str.len().mean())
print('(Answers) Standard deviation of sentence length: ', dataset['answer'].str.len().std())

(Questions) Average sentence length:  17.103324600337082
(Questions) Standard deviation of sentence length:  9.124359472017858
(Answers) Average sentence length:  54.53903092455196
(Answers) Standard deviation of sentence length:  843.2193542807091


### Update the sentences to fixed length
Update all sentences with a fixed size, to process the sentences as batches.

In [11]:
# maximum sentence length
max_sent_length = {'question' : 30, 'answer': 70}

for column in dataset.columns:
    for tokens in dataset[column]: 
        
        # adding the start token
        tokens.insert(0, '<s>')
        
        if len(tokens) >= max_sent_length[column]:
            tokens = tokens[:max_sent_length[column] - 1]
            tokens.append('</s>')
            
        if len(tokens) < max_sent_length[column]:
            tokens.extend(['</s>' for _ in range(max_sent_length[column] - len(tokens))])